# Importing

In [1]:
import pandas as pd
import requests
url = "https://github.com/Trilokesh-Praxis-2023/Data/raw/main/MKTA_Project.xlsx"
df = pd.read_excel(url)
print(df.head())

   Customer_ID  Purchase_Amount Purchase_Date
0          760             25.0    2009-11-06
1          860             50.0    2012-09-28
2         1200            100.0    2005-10-25
3         1420             50.0    2009-07-09
4         1940             70.0    2013-01-25


In [2]:
df1=df.copy()

# Question 1

In [3]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Find the last date in the DataFrame
last_date = df['Purchase_Date'].max()

# Task 1: Feature Engineering - Calculate Recency (R), Frequency (F), and Monetary (M) for each customer
customer_rfm = df.groupby('Customer_ID').agg({
    'Purchase_Date': lambda x: (last_date - x.max()).days,  
    'Customer_ID': 'count',  # Frequency
    'Purchase_Amount': 'sum'  # Monetary
}).rename(columns={
    'Purchase_Date': 'Recency',
    'Customer_ID': 'Frequency',
    'Purchase_Amount': 'Monetary'
}).reset_index()

# Select relevant features for clustering
X = customer_rfm[['Recency', 'Frequency', 'Monetary']]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Use K-Means to cluster the customers into 8 segments
kmeans = KMeans(n_clusters=8, random_state=42)
customer_rfm['Segment'] = kmeans.fit_predict(X_scaled)

# Task 1: Identify the most important segment
segment_importance = customer_rfm.groupby('Segment')['Monetary'].mean()
most_important_segment = segment_importance.idxmax()

print(f"Most important segment: {most_important_segment}")
# Task 2: Identify the second most important segment
segment_importance.drop(most_important_segment, inplace=True)
second_most_important_segment = segment_importance.idxmax()
print(f"Second most important segment: {second_most_important_segment}")

AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Assuming 'customer_rfm' is your DataFrame with Recency, Frequency, and Monetary columns
# Feature Engineering: Calculate Recency (R), Frequency (F), and Monetary (M)
customer_rfm['R_Quartile'] = customer_rfm['Recency'].apply(lambda x: 3 if x <= 365 else (2 if x <= 730 else 1))
customer_rfm['F_Quartile'] = customer_rfm['Frequency'].apply(lambda x: 3 if x >= 3 else (2 if x >= 2 else 1))
customer_rfm['M_Quartile'] = customer_rfm['Monetary'].apply(lambda x: 3 if x >= 45 else (2 if x >= 27.5 else 1))

# Create a 'Segment' column based on quartiles
customer_rfm['Segment'] = customer_rfm['R_Quartile'].astype(str) + customer_rfm['F_Quartile'].astype(str) + customer_rfm['M_Quartile'].astype(str)

# Select relevant features for clustering
X = customer_rfm[['Recency', 'Frequency', 'Monetary']]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Use K-Means to cluster the customers into 8 segments
kmeans = KMeans(n_clusters=8, random_state=42)
customer_rfm['Cluster'] = kmeans.fit_predict(X_scaled)

# Calculate the average monetary value for each segment
average_monetary_by_segment = customer_rfm.groupby('Segment')['Monetary'].mean().reset_index()
print(average_monetary_by_segment)


/Users/trilokesh/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


   Segment    Monetary
0      111   17.032515
1      112   32.583275
2      113  110.271113
3      121   20.896779
4      122   35.697460
5      123  118.625474
6      131   20.300000
7      132   35.516667
8      133  257.849779
9      211   15.646186
10     212   30.257235
11     213  127.981772
12     221   20.615385
13     222   36.361000
14     223  144.802632
15     231   21.666667
16     232   35.555556
17     233  395.570850
18     311   17.360976
19     312   31.672451
20     313  151.579358
21     321   21.250000
22     322   36.525312
23     323  152.633140
24     331   21.600000
25     332   37.403333
26     333  471.937042


In [ ]:
customer_rfm

,Customer_ID,Recency,Frequency,Monetary,Segment,R_Quartile,F_Quartile,M_Quartile,RFMClass,Cluster
0,10,3828,1,30.0,112,1,1,2,112,2
1,80,342,7,500.0,333,3,3,3,333,0
2,90,757,10,1158.0,133,1,3,3,133,5
3,120,1400,1,20.0,111,1,1,1,111,1
4,130,2969,2,100.0,123,1,2,3,123,2
...,...,...,...,...,...,...,...,...,...,...
18412,263820,0,1,10.0,311,3,1,1,311,7
18413,263870,134,1,50.0,313,3,1,3,313,7
18414,263880,33,1,20.0,311,3,1,1,311,7
18415,263890,4,1,54.0,313,3,1,3,313,7


In [ ]:
rfm_mean_by_class = customer_rfm.groupby('Cluster').mean()


/var/folders/3f/y5zf37093hx4z8g6fjspk6080000gn/T/ipykernel_86889/2845513233.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  rfm_mean_by_class = customer_rfm.groupby('Cluster').mean()


In [ ]:
rfm_mean_by_class

,Customer_ID,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile
Cluster,,,,,,,
0,118662.856544,433.584312,5.203859,277.528947,2.368708,2.991611,2.992030
1,132894.861595,1624.631126,1.761534,75.696047,1.000000,1.665778,2.334017
2,78628.714789,2895.620724,1.326962,57.450646,1.000000,1.294014,1.938380
3,99096.923077,502.163462,7.182692,3896.401731,2.461538,2.721154,3.000000
4,108213.684211,361.894737,6.789474,11458.578947,2.631579,2.894737,3.000000
5,68430.598485,273.346212,9.504545,554.858182,2.665152,3.000000,3.000000
6,62554.333333,185.180000,18.580000,862.832333,2.780000,3.000000,3.000000
7,210841.132987,383.230356,1.568105,80.606062,2.295149,1.568105,2.437444


In [ ]:
import pandas as pd

# Create a DataFrame from the provided data
data = {
    "Customer_ID": [118662.856544, 132894.861595, 78628.714789, 99096.923077, 108213.684211, 68430.598485, 62554.333333, 210841.132987],
    "Recency": [433.584312, 1624.631126, 2895.620724, 502.163462, 361.894737, 273.346212, 185.180000, 383.230356],
    "Frequency": [5.203859, 1.761534, 1.326962, 7.182692, 6.789474, 9.504545, 18.580000, 1.568105],
    "Monetary": [277.528947, 75.696047, 57.450646, 3896.401731, 11458.578947, 554.858182, 862.832333, 80.606062],
    "R_Quartile": [2.368708, 1.000000, 1.000000, 2.461538, 2.631579, 2.665152, 2.780000, 2.295149],
    "F_Quartile": [2.991611, 1.665778, 1.294014, 2.721154, 2.894737, 3.000000, 3.000000, 1.568105],
    "M_Quartile": [2.992030, 2.334017, 1.938380, 3.000000, 3.000000, 3.000000, 3.000000, 2.437444],
    "Cluster": [0,1,2,3,4,5,6,7]
}

df = pd.DataFrame(data)

df



,Customer_ID,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,Cluster
0,118662.856544,433.584312,5.203859,277.528947,2.368708,2.991611,2.992030,0
1,132894.861595,1624.631126,1.761534,75.696047,1.000000,1.665778,2.334017,1
2,78628.714789,2895.620724,1.326962,57.450646,1.000000,1.294014,1.938380,2
3,99096.923077,502.163462,7.182692,3896.401731,2.461538,2.721154,3.000000,3
4,108213.684211,361.894737,6.789474,11458.578947,2.631579,2.894737,3.000000,4
5,68430.598485,273.346212,9.504545,554.858182,2.665152,3.000000,3.000000,5
6,62554.333333,185.180000,18.580000,862.832333,2.780000,3.000000,3.000000,6
7,210841.132987,383.230356,1.568105,80.606062,2.295149,1.568105,2.437444,7


In [4]:
import pandas as pd

# Create a DataFrame with the provided data
data = {
    "Customer_ID": [118662.856544, 132894.861595, 78628.714789, 99096.923077, 108213.684211, 68430.598485, 62554.333333, 210841.132987],
    "Recency": [433.584312, 1624.631126, 2895.620724, 502.163462, 361.894737, 273.346212, 185.180000, 383.230356],
    "Frequency": [5.203859, 1.761534, 1.326962, 7.182692, 6.789474, 9.504545, 18.580000, 1.568105],
    "Monetary": [277.528947, 75.696047, 57.450646, 3896.401731, 11458.578947, 554.858182, 862.832333, 80.606062],
    "R_Quartile": [2.368708, 1.000000, 1.000000, 2.461538, 2.631579, 2.665152, 2.780000, 2.295149],
    "F_Quartile": [2.991611, 1.665778, 1.294014, 2.721154, 2.894737, 3.000000, 3.000000, 1.568105],
    "M_Quartile": [2.992030, 2.334017, 1.938380, 3.000000, 3.000000, 3.000000, 3.000000, 2.437444],
    "Cluster": [0,1,2,3,4,5,6,7]
}

df = pd.DataFrame(data)

# Assign weights to recency, frequency, and monetary for marketing logic
recency_weight = 0.4
frequency_weight = 0.3
monetary_weight = 0.3

# Calculate a marketing score for each segment
df['Marketing_Score'] = (
    recency_weight * df['Recency'] +
    frequency_weight * df['Frequency'] +
    monetary_weight * df['Monetary']
)

# Sort the DataFrame by Marketing_Score in descending order
df_sorted = df.sort_values(by='Marketing_Score', ascending=False)

# Top two target segments
top_two_segments = df_sorted.head(2)

print(top_two_segments[['Cluster', 'Marketing_Score']])


   Cluster  Marketing_Score
4        4      3584.368421
3        3      1371.940712


In [ ]:
df_sorted 

In [46]:
import pandas as pd

# Assuming you have a DataFrame 'df' with columns 'Cluster', 'R_Quartile', 'F_Quartile', and 'M_Quartile'

# Calculate a composite score based on RFM quartiles
df['RFM_Score'] = df['R_Quartile'] * 100 + df['F_Quartile'] * 10 + df['M_Quartile']

# Define a function to rate the clusters based on RFM scores
def rate_cluster(rfm_score):
    if rfm_score >= 300:
        return 'High Value'
    elif 200 <= rfm_score < 300:
        return 'Medium Value'
    else:
        return 'Low Value'

# Apply the function to create a new column 'Cluster_Rating'
df['Cluster_Rating'] = df['RFM_Score'].apply(rate_cluster)

# Display the DataFrame with Cluster Ratings
print(df[['Cluster', 'RFM_Score', 'Cluster_Rating']])


   Cluster   RFM_Score Cluster_Rating
0        0  269.778940   Medium Value
1        1  118.991797      Low Value
2        2  114.878520      Low Value
3        3  276.365340   Medium Value
4        4  295.105270   Medium Value
5        5  299.515200   Medium Value
6        6  311.000000     High Value
7        7  247.633394   Medium Value


In [47]:
import pandas as pd

# Assuming you have a DataFrame named 'df' with the provided feature engineering code
# If not, replace 'df' with your actual DataFrame name

# Feature Engineering: Calculate Recency (R), Frequency (F), and Monetary (M) quartiles
df['R_Quartile'] = df['Recency'].apply(lambda x: 3 if x <= 365 else (2 if x <= 730 else 1))
df['F_Quartile'] = df['Frequency'].apply(lambda x: 3 if x >= 3 else (2 if x >= 2 else 1))
df['M_Quartile'] = df['Monetary'].apply(lambda x: 3 if x >= 45 else (2 if x >= 27.5 else 1))

# Create a 'Segment' column based on quartiles
df['Segment'] = df['R_Quartile'].astype(str) + df['F_Quartile'].astype(str) + df['M_Quartile'].astype(str)

# Define a function to rate the cluster based on R, F, M scores
def rate_cluster(row):
    r_score = row['R_Quartile']
    f_score = row['F_Quartile']
    m_score = row['M_Quartile']
    
    # Calculate the score based on the provided marketing logic
    score = (r_score + f_score + m_score) / 3
    
    return round(score * 100, 2)

# Apply the function to create a new 'Score' column
df['Score'] = df.apply(rate_cluster, axis=1)

# Sort the DataFrame based on the 'Score' column in descending order
df_sorted = df.sort_values(by='Score', ascending=False)

# Extract the top two target segments
top_two_segments = df_sorted.head(2)[['Cluster', 'Recency', 'Frequency', 'Monetary', 'Segment', 'Score']]

# Print the top two target segments
print(top_two_segments)


   Cluster     Recency  Frequency      Monetary Segment  Score
4        4  361.894737   6.789474  11458.578947     333  300.0
5        5  273.346212   9.504545    554.858182     333  300.0


In [48]:

df_sorted['Monetary']

,Customer_ID,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,Cluster,Marketing_Score,RFM_Score,Cluster_Rating,Segment,Score
4,108213.684211,361.894737,6.789474,11458.578947,3,3,3,4,3584.368421,295.105270,Medium Value,333,300.00
5,68430.598485,273.346212,9.504545,554.858182,3,3,3,5,278.647303,299.515200,Medium Value,333,300.00
6,62554.333333,185.180000,18.580000,862.832333,3,3,3,6,338.495700,311.000000,High Value,333,300.00
0,118662.856544,433.584312,5.203859,277.528947,2,3,3,0,258.253567,269.778940,Medium Value,233,266.67
3,99096.923077,502.163462,7.182692,3896.401731,2,3,3,3,1371.940712,276.365340,Medium Value,233,266.67
7,210841.132987,383.230356,1.568105,80.606062,2,1,3,7,177.944392,247.633394,Medium Value,213,200.00
1,132894.861595,1624.631126,1.761534,75.696047,1,1,3,1,673.089725,118.991797,Low Value,113,166.67
2,78628.714789,2895.620724,1.326962,57.450646,1,1,3,2,1175.881572,114.878520,Low Value,113,166.67


**Most Important Segment: Segment 2 - Low Recency, High Frequency, High Monetary**
- **Characteristics:** Low recency, high frequency, high monetary values.
- **Average Monetary Value:** $2632.94.

**Why Important:**
1. **High Revenue:** Significant contribution to revenue with frequent and high-value transactions.
2. **Recent Activity:** Despite low recency, ongoing engagement and recent purchases.
3. **Upsell Opportunities:** Potential for upselling higher-value products and cross-selling.

**Marketing Actions:**
1. Targeted promotions for higher spending.
2. Personalized campaigns for enhanced loyalty.
3. Implement loyalty programs for long-term relationships.
4. Explore cross-selling strategies.

**Second Most Important Segment: Segment 7 - Low Recency, Medium Frequency, Low Monetary**
- **Characteristics:** Low recency, medium frequency, low monetary values.
- **Average Monetary Value:** $720.75.

**Why Important:**
1. **Revenue Potential:** Despite lower monetary values, potential for cumulative revenue due to medium frequency.
2. **Re-Engagement:** Strategies to re-engage customers for increased frequency and spending.

**Marketing Actions:**
1. Targeted campaigns for re-engagement.
2. Incentives for increased frequency and spending.
3. Consider promotions to boost average transaction values.

---
---

In [4]:
l=[1,2,2,'u']
len(l)

4

# Question-2

In [31]:
# Feature Engineering: Calculate Recency (R), Frequency (F), and Monetary (M)
customer_rfm['R_Quartile'] = customer_rfm['Recency'].apply(lambda x: 3 if x <= 365 else (2 if x <= 730 else 1))
customer_rfm['F_Quartile'] = customer_rfm['Frequency'].apply(lambda x: 3 if x >= 3 else (2 if x >= 2 else 1))
customer_rfm['M_Quartile'] = customer_rfm['Monetary'].apply(lambda x: 3 if x >= 45 else (2 if x >= 27.5 else 1))

In [32]:
# Assuming 'customer_rfm' is your DataFrame with Recency, Frequency, and Monetary columns
customer_rfm['RFMClass'] = customer_rfm['R_Quartile'].astype(str) + customer_rfm['F_Quartile'].astype(str) + customer_rfm['M_Quartile'].astype(str)

In [7]:
customer_rfm['RFMClass']

0        112
1        333
2        133
3        111
4        123
        ... 
18412    311
18413    313
18414    311
18415    313
18416    313
Name: RFMClass, Length: 18417, dtype: object

In [8]:
customer_rfm.columns

Index(['Customer_ID', 'Recency', 'Frequency', 'Monetary', 'Segment',
       'R_Quartile', 'F_Quartile', 'M_Quartile', 'RFMClass'],
      dtype='object')

In [33]:
customer_rfm['Segment'] = pd.factorize(customer_rfm['Segment'])[0]

In [34]:
customer_rfm.columns

Index(['Customer_ID', 'Recency', 'Frequency', 'Monetary', 'Segment',
       'R_Quartile', 'F_Quartile', 'M_Quartile', 'RFMClass'],
      dtype='object')

In [35]:
# Assuming 'customer_rfm' is your DataFrame with updated column names
customer_rfm['Segment'] = 'Other'  # Default segment

# Update segment based on RFM scores
customer_rfm.loc[customer_rfm['RFMClass'] == '333', 'Segment'] = 'Best Customers'
customer_rfm.loc[customer_rfm['RFMClass'].str.contains('3..'), 'Segment'] = 'Frequent Buyers'
customer_rfm.loc[customer_rfm['RFMClass'].str.contains('3..') & (customer_rfm['RFMClass'] != '333'), 'Segment'] = 'Potential Customer'
customer_rfm.loc[customer_rfm['RFMClass'].str.contains('..3') & (customer_rfm['Recency'] != 1), 'Segment'] = 'Big Spenders'
customer_rfm.loc[customer_rfm['RFMClass'] == '133', 'Segment'] = 'Cold Customers'
customer_rfm.loc[customer_rfm['RFMClass'] == '111', 'Segment'] = 'Inactive Customers'


In [36]:
customer_rfm.groupby('Segment').first()

,Customer_ID,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMClass
Segment,,,,,,,,
Big Spenders,80,342,7,500.0,3,3,3,333
Cold Customers,90,757,10,1158.0,1,3,3,133
Frequent Buyers,4490,1,11,935.0,3,3,3,333
Inactive Customers,120,1400,1,20.0,1,1,1,111
Other,10,3828,1,30.0,1,1,2,112
Potential Customer,2180,133,1,30.0,3,1,2,312


In [13]:
len(customer_rfm.groupby('RFMClass').first())

27

In [14]:
rfm_class_counts = customer_rfm['RFMClass'].value_counts().reset_index()
rfm_class_counts.columns = ['RFMClass', 'Count']


In [15]:
rfm_class_counts = customer_rfm['RFMClass'].value_counts().reset_index().rename(columns={'index': 'RFMClass', 'RFMClass': 'Count'})

In [16]:
rfm_class_counts = rfm_class_counts.sort_values(by='RFMClass')


In [38]:
rfm_mean_by_class = customer_rfm.groupby('RFMClass').mean()
rfm_mean_by_class


/var/folders/3f/y5zf37093hx4z8g6fjspk6080000gn/T/ipykernel_86889/1490082578.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  rfm_mean_by_class = customer_rfm.groupby('RFMClass').mean()


,Customer_ID,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile
RFMClass,,,,,,,
111,121600.324484,2287.689897,1.000000,17.032515,1.0,1.0,1.0
112,135414.817795,2030.902508,1.000000,32.583275,1.0,1.0,2.0
113,141852.186801,1901.172819,1.000000,110.271113,1.0,1.0,3.0
121,106573.624161,1933.221477,2.000000,20.896779,1.0,2.0,1.0
122,100540.484988,2003.099307,2.000000,35.697460,1.0,2.0,2.0
123,103281.321125,1927.803139,2.000000,118.625474,1.0,2.0,3.0
131,85277.000000,1741.800000,3.200000,20.300000,1.0,3.0,1.0
132,88689.833333,1737.883333,3.216667,35.516667,1.0,3.0,2.0
133,76466.251845,1612.451058,4.509100,257.849779,1.0,3.0,3.0


In [17]:
rfm_class_counts

,RFMClass,Count
1,111,2712
2,112,2113
4,113,1788
16,121,149
10,122,433
5,123,1529
23,131,10
18,132,60
3,133,2033
15,211,236


In [18]:
# Assuming 'customer_rfm' is your DataFrame
selected_rfm_values = [121, 131, 133, 223, 233, 222, 232, 113]

# Filter rows with selected RFM values
selected_rows = customer_rfm[customer_rfm['RFMClass'].isin(selected_rfm_values)]

# Sum the values based on RFMClass
rfm_sum = selected_rows.groupby('RFMClass').agg({
    'Recency': 'sum',
    'Frequency': 'sum',
    'Monetary': 'sum'
}).reset_index()

print(rfm_sum)


Empty DataFrame
Columns: [RFMClass, Recency, Frequency, Monetary]
Index: []


In [19]:
# Assuming 'customer_rfm' is your DataFrame with 'RFMScore' column
customer_rfm['Segment'] = 'Other'  # Default segment

# Update segment based on RFM scores
customer_rfm.loc[customer_rfm['RFMScore'] == '333', 'Segment'] = 'Best Customers'
customer_rfm.loc[customer_rfm['RFMScore'].str.contains('3..'), 'Segment'] = 'Frequent Buyers'
customer_rfm.loc[customer_rfm['RFMScore'].str.contains('3..') & (customer_rfm['RFMScore'] != '333'), 'Segment'] = 'Potential Customer'
customer_rfm.loc[customer_rfm['RFMScore'].str.contains('..3') & (customer_rfm['Recency'] != 1), 'Segment'] = 'Big Spenders'
customer_rfm.loc[customer_rfm['RFMScore'] == '133', 'Segment'] = 'Cold Customers'
customer_rfm.loc[customer_rfm['RFMScore'] == '111', 'Segment'] = 'Inactive Customers'


KeyError: 'RFMScore'